In [3]:
import cv2 
#cv2 is used for OpenCV library
#img = cv2.imread("image1.jpeg")
img = cv2.imread("img1.jpg")
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
(thresh, binary) = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
cv2.imshow("Black and White Image",binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
import random
import cv2

def add_noise(img):

	# Getting the dimensions of the image
	row , col = img.shape
	
	# Randomly pick some pixels in the
	# image for coloring them white
	# Pick a random number between 300 and 10000
	number_of_pixels = random.randint(300, 10000)
	for i in range(number_of_pixels):
		
		# Pick a random y coordinate
		y_coord=random.randint(0, row - 1)
		
		# Pick a random x coordinate
		x_coord=random.randint(0, col - 1)
		
		# Color that pixel to white
		img[y_coord][x_coord] = 255
		
	# Randomly pick some pixels in
	# the image for coloring them black
	# Pick a random number between 300 and 10000
	number_of_pixels = random.randint(300 , 10000)
	for i in range(number_of_pixels):
		
		# Pick a random y coordinate
		y_coord=random.randint(0, row - 1)
		
		# Pick a random x coordinate
		x_coord=random.randint(0, col - 1)
		
		# Color that pixel to black
		img[y_coord][x_coord] = 0
		
	return img

# salt-and-pepper noise can
# be applied only to grayscale images
# Reading the color image in grayscale image
#img = cv2.imread('lena.jpg',				cv2.IMREAD_GRAYSCALE)

#Storing the image
image=add_noise(binary)
cv2.imwrite('salt-and-pepper-added.jpg',image)


True

In [6]:
import numpy
from PIL import Image


def median_filter(data, filter_size):
    temp = []
    indexer = filter_size // 2
    data_final = []
    data_final = numpy.zeros((len(data),len(data[0])))
    for i in range(len(data)):

        for j in range(len(data[0])):

            for z in range(filter_size):
                if i + z - indexer < 0 or i + z - indexer > len(data) - 1:
                    for c in range(filter_size):
                        temp.append(0)
                else:
                    if j + z - indexer < 0 or j + indexer > len(data[0]) - 1:
                        temp.append(0)
                    else:
                        for k in range(filter_size):
                            temp.append(data[i + z - indexer][j + k - indexer])

            temp.sort()
            data_final[i][j] = temp[len(temp) // 2]
            temp = []
    return data_final


def main():
    #img = Image.open("noisyimg.png").convert("L")
    arr = numpy.array(image)
    removed_noise = median_filter(arr, 3) 
    finimg = Image.fromarray(removed_noise)
    finimg.show()
    cv2.imwrite('salt-and-pepper-rmdmedian.jpg',removed_noise)


main()

In [7]:
Opencv_Median = cv2.medianBlur(image, 3)
cv2.imwrite('salt-and-pepper-removed2.jpg',Opencv_Median)

True

In [9]:
psnr = cv2.PSNR(image, Opencv_Median)
print(psnr)

10.9489800216838


In [7]:
import numpy as np
from PIL import Image

# Load the image
image = Image.open('img1.jpg')

# Convert the image to a numpy array
image_array = np.array(image)

# Calculate the standard deviation of the noise
sigma = 10

# Create a noise array with the same shape as the image
noise = np.random.normal(scale=sigma, size=image_array.shape)

# Add the noise to the image
noisy_image_array = image_array + noise

# Clip the pixel values to the valid range [0, 255]
noisy_image_array = np.clip(noisy_image_array, 0, 255)

# Convert the numpy array back to an image
noisy_image = Image.fromarray(noisy_image_array.astype(np.uint8))

# Save the noisy image
noisy_image.save('gaussian_noisy_image.jpg')


In [9]:
import numpy as np
from scipy.signal import wiener
from PIL import Image

# Load the image
image = Image.open('gaussian_noisy_image.jpg')

# Convert the image to a numpy array
image_array = np.array(image)

# Extract the individual color channels
red_channel = image_array[:, :, 0]
green_channel = image_array[:, :, 1]
blue_channel = image_array[:, :, 2]

# Set the kernel size
kernel_size = (3, 3)

# Apply the Wiener filter to each channel separately
filtered_red_channel = wiener(red_channel, mysize=kernel_size)
filtered_green_channel = wiener(green_channel, mysize=kernel_size)
filtered_blue_channel = wiener(blue_channel, mysize=kernel_size)

# Combine the filtered channels into a single image
filtered_image_array = np.stack((filtered_red_channel, filtered_green_channel, filtered_blue_channel), axis=2)

# Convert the filtered numpy array back to an image
filtered_image = Image.fromarray(np.uint8(filtered_image_array))

# Save the filtered image
filtered_image.save('gaussianrmdwiener_image.jpg')


In [14]:
pip install Pillow scipy

Note: you may need to restart the kernel to use updated packages.


In [11]:
filt_arr = np.array(filtered_image)
original = np.array(image_array)
psnr = cv2.PSNR(original, filt_arr)
print(psnr)

20.160015051034073


In [12]:
import numpy as np
import cv2

def add_poisson_noise(image_array, scale):
    """
    Add Poisson noise to an image.
    Args:
        image_array (numpy array): Input image.
        scale (int): Scaling factor for Poisson noise.
    Returns:
        numpy array: Image with Poisson noise added.
    """
    # Generate Poisson noise
    noisy = np.random.poisson(image_array * scale) / float(scale)
    
    # Clip values to ensure they are within the valid range for the image data type
    noisy = np.clip(noisy, 0, 255)
    
    # Convert the data type back to the original
    noisy = noisy.astype(image_array.dtype)
    
    return noisy

# Load the image
img_array = cv2.imread("img1.jpg")

# Apply Poisson noise
noisy_img_array = add_poisson_noise(img_array, scale=50)

# Display the noisy image
cv2.imshow("poisson Image", noisy_img_array)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('poisson_added.jpg',noisy_img_array)


True

In [22]:
import cv2
import numpy as np
from scipy.signal import wiener

# Load the image
img = cv2.imread('img1.jpg', 0)

# Add Poisson noise to the image
poisson_noise = np.random.poisson(lam=img, size=img.shape)
noisy_img = img + poisson_noise.astype(np.uint8)

# Apply Wiener filter to remove Poisson noise
filtered_img = wiener(noisy_img, mysize=None, noise=None)

# Convert the filtered image back to uint8 data type
filtered_img = filtered_img.astype(np.uint8)

# Display the original, noisy, and filtered images
cv2.imshow('Original Image', img)
cv2.imshow('Noisy Image', noisy_img)
cv2.imshow('Filtered Image', filtered_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('Poisson Original.jpg',img)
cv2.imwrite('Poissonadded noised.jpg',noisy_img)
cv2.imwrite('Poissonrmdwiener Filtered.jpg',filtered_img)


True

In [23]:
image_array = np.array(filtered_img)
original = np.array(img)
psnr = cv2.PSNR(original, image_array.astype(np.uint8))
print(psnr)

9.648915599273263


In [13]:
import cv2
import numpy as np

# Load an image
img = cv2.imread('img1.jpg')

# Add speckle noise to the image
noise = np.random.normal(0, 1, img.shape)
noisy_img_speckle = img + img * noise

# Display the noisy image
cv2.imshow('Noisy Image speckle', noisy_img_speckle)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite(' speckle_added_formean.jpg',noisy_img_speckle)


True

In [14]:
import cv2
import numpy as np

# Read the image
img = cv2.imread('img1.jpg')

# Add speckle noise
speckle_noise = np.random.randn(*img.shape) * 0.5
speckled_img = img + img * speckle_noise

# Define kernel size for mean filter
kernel_size = 5

# Apply mean filter to remove speckle noise
filtered_img = cv2.blur(speckled_img, (kernel_size, kernel_size))

# Display original and filtered images
cv2.imshow('Original', img)
cv2.imshow('Speckled', speckled_img)
cv2.imshow('Filtered', filtered_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('specklermdmean.jpg',noisy_img_array)


True

In [15]:
filt_arr = np.array(noisy_img_array)
original = np.array(img)
psnr = cv2.PSNR(original, filt_arr)
print(psnr)

3.8387141854498994


In [16]:
import cv2
import numpy as np
# Read the image
img7 = cv2.imread('img1.jpg', cv2.IMREAD_GRAYSCALE)

# Add speckle noise
noise = np.random.normal(0, 1, img7.shape)
noisy_img7 = img7 + img7 * noise
cv2.imwrite('speckle-noise1.jpg',noisy_img7)


# Apply Gaussian filter with kernel size 5x5 and standard deviation 1.5
gaussian = cv2.GaussianBlur(noisy_img7, (5, 5), 1.5)

# Display the original and filtered images
cv2.imshow('Originalg Image', img7)
cv2.imshow('Noisyg Image', noisy_img7)
cv2.imshow('Filteredg Image', gaussian)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('Speckledrmdg Original.jpg',img7)
cv2.imwrite('Speckledrmdg noised.jpg',noisy_img7)
cv2.imwrite('Speckledrmdg Filtered.jpg',gaussian)



True

In [17]:
image_array = np.array(gaussian)
original = np.array(img7)
psnr = cv2.PSNR(original, image_array.astype(np.uint8))
print(psnr)

11.260750071973447


In [19]:
import cv2
import numpy as np

# Load image
img7 = cv2.imread('img1.jpg', cv2.IMREAD_GRAYSCALE)

# Add speckle noise
noise = np.random.normal(0, 1, img7.shape)
noisy_img7 = img7 + img7 * noise
cv2.imwrite('speckle-noise-added-for-median.jpg',noisy_img7)
# Apply median filter to remove noise
filtered_img7 = cv2.medianBlur(noisy_img7.astype(np.uint8), 3)

# Display original, noisy, and filtered images
#cv2.imwrite('gaussian-noise-3.jpg',noisy_img3)
cv2.imwrite('speckle-noise-removed-by-median-filter.jpg',filtered_img7)

True

In [20]:
image_array = np.array(filtered_img7)
original = np.array(img7)
psnr = cv2.PSNR(original, image_array.astype(np.uint8))
print(psnr)

10.962046888003753
